In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoConfig
import pandas as pd
import sys
sys.path.insert(1, 'scripts')

In [2]:
config = AutoConfig.from_pretrained('model/for_claims/checkpoint/train-300-t1')
tokenizer = AutoTokenizer.from_pretrained(
    'model/for_claims/checkpoint/train-300-t1')
model = AutoModelForSeq2SeqLM.from_pretrained(
    'model/for_claims/checkpoint/train-300-t1')


In [3]:
# from load_data import find_xml_files, load_objects, load_data
# data = load_data()

In [4]:
data = pd.read_csv(
    'data/claims/evaluation-auto-driving.csv')


In [5]:
data.text[1], data.claim[1]

("Its a no brainer for the following reasons:\nI can make use of the time I spend driving. I spend almost 2 hrs a day driving back and forth from work, or just picking up people. So i can basically send my car to airport to pick up friends or family, the car can drop me to work and go back home or pickup kids.I am a bad driver, been into a few fender benders and my insurance is expected to get sky-high. No driving means I don't have to worry about insurance, even license for that instance. I cannot get a ticket or DUI problem. I don't have to worry about falling asleep on the wheel when I am driving late in the night or making a road trip. I can do what I want. Finding parking is a pain in the cities. Especially when I am running late for the meetings, I spend at least 20 min finding parking spots and then walking all the way back to the office. At the least, I can take calls, attend meetings or even watch movies while driving.\nInitially the investment might be big, but the ROI is pre

In [10]:
inputs = tokenizer(data.text.to_list()[1], max_length=300, return_tensors='pt', padding=True, truncation=True)
summary_ids = model.generate(inputs["input_ids"])
# output = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True)


["Its a no brainer for the following reasons\nI don't have to worry about insurance, even license for that instance.\nI cannot get a ticket or DUI problem.\nOnly problem will exist when I am running late to work and there is a traffic problem\nI hope there would be no accidents to cause delays."]

In [8]:
new_df = pd.DataFrame({
    'text' : data.text.tolist(),
    'ground-truth' : data.claim.tolist(),
    'prediction' : output
})

# for i,j,k in zip(data.text.tolist(), data.claim.tolist(), output):
#     print(i)
#     print()
#     print(j)
#     print()
#     print(k)
    

In [9]:
pd.set_option('display.max_colwidth', None)
new_df.head(2)

,text,ground-truth,prediction
0,"Yes, you would.\nWait, how dare I answer for you? Sure, I can answer for myself, I have been in them many times, from several different teams. But I've also watched other people in them, and seen the results of research at Stanford testing how people react to them.\nAnd the research shows people are amazingly trusting of them. One might even say disturbingly trusting, since all the vehicles today are prototypes which are not rated safe enough to be sold to ""civilians."" People trust them more than the engineers building them would trust them, more than they should. In the Stanford research, they made a fake self-driving car which was actually being driven by the ""observer"" in the back seat with hidden controls. They simulated the vehicle having a system failure and needing immediate take-over. Even after the failure, subjects were happy to push the button to re-engage when told to by the observer, and they would pick up their newspaper and read again.\nSo while yes, you may personally believe you would not get in one, and might even be right about yourself, the reality is huge numbers of people are ready to trust the vehicles, and a few positive experiences make them even more willing. And that's before they are actually ready, and are always operated with safety drivers ready to handle any problem. In truth, even people who say they would not be willing to trust it seem often to change their minds, so you might even be wrong about yourself when you say you would not ride it.\nWould you get into a self-driving elevator? Some day people will think this question is similar to that.","Yes, you would.","Yes\nthe research shows people are amazingly trusting of them. \nSo while yes, you may personally believe you would not get in one, and might even be right about yourself."
1,"Its a no brainer for the following reasons:\nI can make use of the time I spend driving. I spend almost 2 hrs a day driving back and forth from work, or just picking up people. So i can basically send my car to airport to pick up friends or family, the car can drop me to work and go back home or pickup kids.I am a bad driver, been into a few fender benders and my insurance is expected to get sky-high. No driving means I don't have to worry about insurance, even license for that instance. I cannot get a ticket or DUI problem. I don't have to worry about falling asleep on the wheel when I am driving late in the night or making a road trip. I can do what I want. Finding parking is a pain in the cities. Especially when I am running late for the meetings, I spend at least 20 min finding parking spots and then walking all the way back to the office. At the least, I can take calls, attend meetings or even watch movies while driving.\nInitially the investment might be big, but the ROI is pretty awesome and worth it.\nOnly problem will exist when I am running late to work and there is a traffic problem, however if everyone is driving a robotics car, i hope there would be no accidents to cause delays.","Its a no brainer\nInitially the investment might be big, but the ROI is pretty awesome and worth it.","Its a no brainer for the following reasons\nI don't have to worry about insurance, even license for that instance.\nI cannot get a ticket or DUI problem.\nOnly problem will exist when I am running late to work and there is a traffic problem\nI hope there would be no accidents to cause delays."
